In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functions import *
from jacobi import *

In [10]:
%load_ext autoreload
%autoreload 2

In [35]:
%%bash
mkdir -p ../build
cd ../build
cmake ../
cmake --build . -j4
cd -

-- HDF5 CXX compiler wrapper is unable to compile a minimal HDF5 program.
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_HDF5Writer.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_filehandler.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_parser.cpp
-- Test source: /home/balazs/Desktop/University/Research/Practice/Laplace/test/test_utilities.cpp
-- Configuring done (0.8s)
-- Generating done (0.0s)
-- Build files have been written to: /home/balazs/Desktop/University/Research/Practice/Laplace/build
[  8%] Built target gtest
[ 12%] Building CXX object src/CMakeFiles/laplace_lib.dir/parser.cpp.o
[ 16%] Building CXX object src/CMakeFiles/laplace_lib.dir/calculator.cpp.o
[ 25%] Built target gmock
[ 33%] Built target gtest_main
[ 37%] Building CXX object src/CMakeFiles/laplace_lib.dir/filehandler.cpp.o
[ 45%] Built target gmock_main
[ 50%] Building CXX object src/CMakeFiles/la

In [36]:
%%bash
cd ../build/
ctest --output-on-failure
cd -

Test project /home/balazs/Desktop/University/Research/Practice/Laplace/build
      Start  1: HDF5WriterTest.WritesParametersCorrectly


 1/13 Test  #1: HDF5WriterTest.WritesParametersCorrectly ......   Passed    0.02 sec
      Start  2: HDF5WriterTest.WritesResultDatasetCorrectly
 2/13 Test  #2: HDF5WriterTest.WritesResultDatasetCorrectly ...   Passed    0.02 sec
      Start  3: HDF5WriterTest.CreatesExpectedFileStructure
 3/13 Test  #3: HDF5WriterTest.CreatesExpectedFileStructure ...   Passed    0.01 sec
      Start  4: HDF5WriterTest.OverwritesFileOnCreation
 4/13 Test  #4: HDF5WriterTest.OverwritesFileOnCreation .......   Passed    0.01 sec
      Start  5: ParserTest.HelpMessageReturnsFalse
 5/13 Test  #5: ParserTest.HelpMessageReturnsFalse ............   Passed    0.00 sec
      Start  6: ParserTest.MissingRequiredArgsThrows
 6/13 Test  #6: ParserTest.MissingRequiredArgsThrows ..........   Passed    0.00 sec
      Start  7: ParserTest.ParsesRequiredArguments
 7/13 Test  #7: ParserTest.ParsesRequiredArguments ............   Passed    0.01 sec
      Start  8: ParserTest.ParsesAllArguments
 8/13 Test  #8: ParserTest.P

In [23]:
generate_ex1(10, 15, "Data")

In [37]:
!time ../build/src/laplace --config ../examples/config.txt --result-path ../examples/Data/N10_M15_res_serial

Laplace equation solver
The used parameters are:
	N: 10
	tol: 0.001
	max iter: 500
	initial state path: ../examples/Data/N10_M15_data.csv
	mask path: ../examples/Data/N10_M15_mask.csv
	result path: ../examples/Data/N10_M15_res_serial
	output: 2
	config path: ../examples/config.txt
Calculation finished after 55 iterations, the result is:
1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
1, 0.484968, 0.273602, 0.169218, 0.109658, 0.072407, 0.0479355, 0.0315136, 0.0204451, 0.0130233, 0.0080956, 0.0048491, 0.00269594, 0.00119569, 0
1, 0.666651, 0.440947, 0.294612, 0.198191, 0.133306, 0.0890853, 0.0588566, 0.0382857, 0.0244241, 0.0151949, 0.00910603, 0.00506383, 0.00224628, 0
1, 0.741406, 0.530285, 0.37197, 0.257413, 0.175926, 0.118622, 0.0787617, 0.0513783, 0.0328283, 0.0204425, 0.0122571, 0.00681833, 0.00302494, 0
1, 0.769653, 0.568651, 0.408102, 0.286563, 0.197582, 0.133915, 0.089184, 0.0582777, 0.0372744, 0.0232244, 0.01393, 0.00775033, 0.00343882, 0
1, 0.769653, 0.568651, 0.408102, 0.286563,

In [24]:
!time python jacobi.py --config config.txt --result-path ../examples/Data/N10_M15_res_python.csv

Laplace equation solver
The used parameters are:
	N: 10
	tol: 0.001
	max iter: 500
	initial state path: ../examples/Data/N10_M15_data.csv
	result path: ../examples/Data/N10_M15_res_python.csv
	mask path: ../examples/Data/N10_M15_mask.csv
	output: 2
	config path: config.txt
Calulation finished after 55 iterations, the result is:
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [1.         0.48496789 0.27360205 0.16921814 0.10965825 0.07240704
  0.04793548 0.03151363 0.02044507 0.01302333 0.0080956  0.0048491
  0.00269594 0.00119569 0.        ]
 [1.         0.666651   0.4409467  0.29461219 0.19819095 0.1333057
  0.08908532 0.05885656 0.0382857  0.02442409 0.01519488 0.00910603
  0.00506383 0.00224628 0.        ]
 [1.         0.74140569 0.53028459 0.37197032 0.25741348 0.17592622
  0.11862171 0.07876175 0.05137831 0.03282834 0.02044248 0.01225705
  0.00681833 0.00302494 0.        ]


In [34]:
!clang-format -i ../src/*.cpp ../src/*.hpp ../test/*.cpp